In [5]:
import numpy as np
import pandas as pd

# CARGAR DATA

In [6]:
spotify_full = pd.read_csv("data/songs_normalize.csv")
df_artists = pd.read_csv("data/Artistas completo.csv")

# LIMPIEZA

Limpieza de la columna genre. Necesitamos obtener solo un genero para una futura clasificacion por id, por lo que hemos optado por quedarnos solo con el primer genero.

In [7]:
spotify_full['genre'] = spotify_full['genre'].str.split(', ')
spotify_full['genre'] = spotify_full['genre'].str[0]

Creación del DF caracteristicas con las columnas correspondientes. Porteriormente creamos la columna caracteristicas_id.

In [8]:
df_caracteristicas = spotify_full[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]
df_caracteristicas['caracteristicas_id'] = df_caracteristicas.index + 1

C:\Users\luisf\AppData\Local\Temp\ipykernel_21140\2160475493.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_caracteristicas['caracteristicas_id'] = df_caracteristicas.index + 1


Creamos el DF de generos musicales. Convertimos valores a lista, hacemos un explode para separarlos generos y creamos una columna para el id.

In [9]:
df_genres = spotify_full[['genre']]
df_genres['genre'] = df_genres['genre'].str.split(', ')
df_genres_explode = df_genres.explode('genre')
df_genres_final = df_genres_explode.drop_duplicates().reset_index(drop= True)
df_genres_final['genre_id'] = df_genres_final.index + 1

C:\Users\luisf\AppData\Local\Temp\ipykernel_21140\706322358.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genres['genre'] = df_genres['genre'].str.split(', ')


Creamos Df de canciones. Nos quedamos con las columnas que nos interesan para las tablas y añadimos la coumna para el id.

In [10]:
df_songs = spotify_full[['song', 'year', 'duration_ms', 'explicit', 'popularity', 'artist', 'genre']]
df_songs['song_id'] = df_songs.index + 1

C:\Users\luisf\AppData\Local\Temp\ipykernel_21140\2326880774.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_songs['song_id'] = df_songs.index + 1


Añadimos una columna de id al DF de artistas completo.

In [11]:
df_artists['artist_id'] = df_artists.index + 1
df_artists.rename(columns= {'name' : 'artist'}, inplace= True)

# AGRUPACIONES

Hacemos un merge entre el Df de canciones y el de artistas para que haya relacion entre los id de artistas y la columna de artistas de songs. Despues nos deshacemos de las columnas sobrantes.

In [12]:
df_songs_final = df_songs.merge(df_artists, on= 'artist', how= 'left')
df_songs_final.drop(['artist', 'type', 'nationality', 'year_of_birth_or_formation'], axis= 1, inplace= True)

Hacemos otro merge para añadir la realcion entre genre:id y genres, asi actualizamos una vez mas el df final de canciones.

In [13]:
df_songs_final = df_songs_final.merge(df_genres_final, on= 'genre', how= 'left')
df_songs_final.drop('genre', axis= 1, inplace= True)

Añadimos una columna para caracteristicas_id, no es encesario hacer un merge, ya que las key de caracteristicas y songs son idénticas.

In [14]:
df_songs_final['caracteristicas_id'] = df_songs.index + 1

# EXPORTAR

Finalmente exportamos los cuatro DF a csv para continuar trabajando en MySQL.

In [15]:
df_songs_final.to_csv('songs.csv', index= False)
df_artists.to_csv('artist.csv', index= False)
df_genres_final.to_csv('genres.csv', index= False)
df_caracteristicas.to_csv('caracteristics.csv', index= False)